<a href="https://colab.research.google.com/github/MayssenBHA/Web-Crawlling/blob/main/Web_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nest_asyncio

In [2]:
!pip install weasyprint
!pip install markdown2



In [3]:
!pip install -U crawl4ai

In [4]:
import crawl4ai
print(crawl4ai.__version__.__version__)

0.7.0


In [5]:
!crawl4ai-setup

[INIT].... → Running post-installation setup... 
[INIT].... → Installing Playwright browsers... 
Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does 

In [6]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check... 
[INIT].... → Crawl4AI 0.7.0 
[TEST].... ℹ Testing crawling capabilities... 
[EXPORT].. ℹ Exporting media (PDF/MHTML/screenshot) took 1.49s 
[FETCH]... ↓ https://crawl4ai.com                                               
| ✓ | ⏱: 3.15s 
[SCRAPE].. ◆ https://crawl4ai.com                                               
| ✓ | ⏱: 0.04s 
[COMPLETE] ● https://crawl4ai.com                                               
| ✓ | ⏱: 3.20s 
[COMPLETE] ● ✅ Crawling test passed! 


In [7]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [8]:
from playwright.async_api import async_playwright

In [9]:
import asyncio
from playwright.async_api import async_playwright

async def test_browser():
    async with async_playwright() as p:
        # Use headless mode (essential for server/cloud environments)
        browser = await p.chromium.launch(headless=True)

        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
            viewport={'width': 1280, 'height': 800}
        )
        page = await context.new_page()
        await page.goto('https://books.toscrape.com', timeout=60000)

        print(f'Title: {await page.title()}')

        await browser.close()

# Run the function
asyncio.run(test_browser())


Title: All products | Books to Scrape - Sandbox


In [10]:
from crawl4ai import AsyncWebCrawler, CacheMode, BrowserConfig, CrawlerRunConfig
async def simple_crawl():
    # Create a crawler configuration that bypasses any cache and always fetches fresh content
    crawler_run_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)

    # Use an asynchronous context manager to launch the web crawler
    async with AsyncWebCrawler() as crawler:
        # Run the asynchronous crawl on the specified URL with the given configuration
        result = await crawler.arun(
            url="https://books.toscrape.com",
            config=crawler_run_config
        )

        # Print the first 500 characters of the raw markdown result (newlines replaced with --)
        print(result.markdown.raw_markdown[:10000].replace("\n", " -- "))

# Execute the simple_crawl async function
asyncio.run(simple_crawl())

[INIT].... → Crawl4AI 0.7.0 

[FETCH]... ↓ https://books.toscrape.com                                                                           |
✓ | ⏱: 1.21s 

[SCRAPE].. ◆ https://books.toscrape.com                                                                           |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://books.toscrape.com                                                                           |
✓ | ⏱: 1.25s 

[Books to Scrape](https://books.toscrape.com/index.html) We love being scraped! --   * [Home](https://books.toscrape.com/index.html) --   * All products --  --  --   * [ Books ](https://books.toscrape.com/catalogue/category/books_1/index.html) --     * [ Travel ](https://books.toscrape.com/catalogue/category/books/travel_2/index.html) --     * [ Mystery ](https://books.toscrape.com/catalogue/category/books/mystery_3/index.html) --     * [ Historical Fiction ](https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html) --     * [ Sequential Art ](https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html) --     * [ Classics ](https://books.toscrape.com/catalogue/category/books/classics_6/index.html) --     * [ Philosophy ](https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html) --     * [ Romance ](https://books.toscrape.com/catalogue/category/books/romance_8/index.html) --     * [ Womens Fiction ](https://books.toscrap

In [11]:
# Define an asynchronous function to crawl a page with dynamic content
async def crawl_dynamic_content():
    # You can wait for a condition in the page before extracting content using `wait_for`.
    # Option 1: Wait for at least 10 articles to appear on the page (JavaScript condition)
    # wait_for = """() => {
    #     return Array.from(document.querySelectorAll('article.tease-card')).length > 10;
    # }"""

    # Option 2: Wait until the 10th article element is present (CSS selector)
    # wait_for = "article.tease-card:nth-child(10)"

    # Start the AsyncWebCrawler context
    async with AsyncWebCrawler() as crawler:
        # JavaScript code to simulate clicking the "Load More" button (if present)
        js_code = [
            "const loadMoreButton = Array.from(document.querySelectorAll('button')).find(button => button.textContent.includes('Load More')); loadMoreButton && loadMoreButton.click();"
        ]

        # Define the crawling configuration
        config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,  # Use cached content if available
            js_code=js_code,               # Inject custom JavaScript into the page before extraction
            # wait_for=wait_for,          # Optional: Uncomment to wait for specific content to load
        )

        # Run the crawler on the target URL with the specified configuration
        result = await crawler.arun(
            url="https://books.toscrape.com",
            config=config,
        )

        # Print the first 500 characters of the extracted raw markdown, replacing line breaks with '--'
        print(result.markdown.raw_markdown[:500].replace("\n", " -- "))

# Execute the async function
asyncio.run(crawl_dynamic_content())

[INIT].... → Crawl4AI 0.7.0 

[FETCH]... ↓ https://books.toscrape.com                                                                           |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://books.toscrape.com                                                                           |
✓ | ⏱: 0.01s 

[Books to Scrape](https://books.toscrape.com/index.html) We love being scraped! --   * [Home](https://books.toscrape.com/index.html) --   * All products --  --  --   * [ Books ](https://books.toscrape.com/catalogue/category/books_1/index.html) --     * [ Travel ](https://books.toscrape.com/catalogue/category/books/travel_2/index.html) --     * [ Mystery ](https://books.toscrape.com/catalogue/category/books/mystery_3/index.html) --     * [ Historical Fiction ](https://books.toscrape.com/catalogue/category/books/historic


In [12]:
# Define an asynchronous function to analyze links on a webpage
async def link_analysis():
    # Start the AsyncWebCrawler context
    async with AsyncWebCrawler() as crawler:
        # Configure the crawler to:
        # - Use cache if available
        # - Exclude external links (outside the domain)
        # - Exclude links to social media platforms
        # Optionally, you can explicitly exclude specific domains like facebook.com or twitter.com
        config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,
            exclude_external_links=True,
            exclude_social_media_links=True,
            # exclude_domains=["facebook.com", "twitter.com"]  # Optional domain exclusion
        )

        # Run the crawler on the target URL with the specified configuration
        result = await crawler.arun(
            url="https://books.toscrape.com",
            config=config,
        )

        # Print the number of internal and external links found
        print(f"Found {len(result.links['internal'])} internal links")
        print(f"Found {len(result.links['external'])} external links")

        # Print the first 5 internal links with their href and visible text
        for link in result.links['internal'][:5]:
            print(f"Href: {link['href']}\nText: {link['text']}\n")

# Run the asynchronous link analysis function
asyncio.run(link_analysis())

[INIT].... → Crawl4AI 0.7.0 

[FETCH]... ↓ https://books.toscrape.com                                                                           |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://books.toscrape.com                                                                           |
✓ | ⏱: 0.01s 

Found 73 internal links
Found 0 external links
Href: https://books.toscrape.com/index.html
Text: Books to Scrape

Href: https://books.toscrape.com/catalogue/category/books_1/index.html
Text: Books

Href: https://books.toscrape.com/catalogue/category/books/travel_2/index.html
Text: Travel

Href: https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
Text: Mystery

Href: https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
Text: Historical Fiction



In [13]:
async def media_handling():
  async with AsyncWebCrawler() as crawler:
    config= CrawlerRunConfig(
        cache_mode=CacheMode.ENABLED,
        exclude_external_images=False,
    )
    result= await crawler.arun(
        url="https://books.toscrape.com",
        config=config,
    )
    for img in result.media['images'][:20]:
      print(f"Image URL: {img['src']}, Alt: {img['alt']}, Score: {img['score']}")

asyncio.run(media_handling())


[INIT].... → Crawl4AI 0.7.0 

[FETCH]... ↓ https://books.toscrape.com                                                                           |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://books.toscrape.com                                                                           |
✓ | ⏱: 0.01s 

Image URL: media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg, Alt: A Light in the Attic, Score: 3
Image URL: media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg, Alt: Tipping the Velvet, Score: 3
Image URL: media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg, Alt: Soumission, Score: 3
Image URL: media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg, Alt: Sharp Objects, Score: 3
Image URL: media/cache/be/a5/bea5697f2534a2f86a3ef27b5a8c12a6.jpg, Alt: Sapiens: A Brief History of Humankind, Score: 3
Image URL: media/cache/68/33/68339b4c9bc034267e1da611ab3b34f8.jpg, Alt: The Requiem Red, Score: 3
Image URL: media/cache/92/27/92274a95b7c251fea59a2b8a78275ab4.jpg, Alt: The Dirty Little Secrets of Getting Your Dream Job, Score: 3
Image URL: media/cache/3d/54/3d54940e57e662c4dd1f3ff00c78cc64.jpg, Alt: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull, Score: 3
Image URL: media/cache/66/88/66883b91f6804b2323c8369331cb7dd1.jpg, Alt: The Boys in th

In [15]:
import asyncio
import csv
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig
from datetime import datetime
from bs4 import BeautifulSoup

async def get_page_books(url, crawler):
    """Extract books from a single page using crawl4ai + BeautifulSoup."""
    result = await crawler.arun(
        url=url,
        config=CrawlerRunConfig(
            cache_mode=CacheMode.BYPASS,
        ),
        max_depth=2
    )

    soup = BeautifulSoup(result.html, 'html.parser')
    books = []

    for book in soup.find_all('article', class_='product_pod'):
        title = book.h3.a['title']
        price = book.find('p', class_='price_color').get_text(strip=True)
        availability = book.find('p', class_='instock').get_text(strip=True)
        rating = book.p['class'][1] if 'class' in book.p.attrs else 'N/A'
        image_url = book.img['src'] if book.img else 'N/A'
        book_url = book.h3.a['href'] if book.h3 else 'N/A'

        # Convert relative URLs to absolute
        base_url = "https://books.toscrape.com/"
        if image_url.startswith('../../'):
            image_url = base_url + image_url.replace('../../', 'media/catalogue/')
        if book_url.startswith('catalogue/'):
            book_url = base_url + book_url
        elif book_url.startswith('../../'):
            book_url = base_url + 'catalogue/' + book_url.replace('../../', '')

        books.append({
            'Title': title,
            'Price': price,
            'Availability': availability,
            'Rating': rating,
            'Image URL': image_url,
            'Book URL': book_url
        })

    return books, soup

async def crawl_all_books(max_pages=5):
    """Crawl multiple pages with controlled depth."""
    csv_filename = f"books_toscrape.csv"

    async with AsyncWebCrawler() as crawler:
        all_books = []
        current_url = "https://books.toscrape.com/"
        pages_crawled = 0

        while current_url and pages_crawled < max_pages:
            print(f"📖 Crawling page {pages_crawled + 1}: {current_url}")
            books, soup = await get_page_books(current_url, crawler)
            all_books.extend(books)
            pages_crawled += 1

            # Find next page
            next_button = soup.find('li', class_='next')
            current_url = None
            if next_button and next_button.a:
                next_page = next_button.a['href']
                if not next_page.startswith('catalogue/'):
                    next_page = f"catalogue/{next_page}"
                current_url = f"https://books.toscrape.com/{next_page}"

        # Save to CSV
        if all_books:
            with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=[
                    'Title', 'Price', 'Availability', 'Rating', 'Image URL', 'Book URL'
                ])
                writer.writeheader()
                writer.writerows(all_books)
            print(f"✅ Saved {len(all_books)} books from {pages_crawled} pages to {csv_filename}")
        else:
            print("❌ No books found!")

# Run with max 3 pages (adjust as needed)
asyncio.run(crawl_all_books(max_pages=3))

[INIT].... → Crawl4AI 0.7.0 

📖 Crawling page 1: https://books.toscrape.com/


[FETCH]... ↓ https://books.toscrape.com/                                                                          |
✓ | ⏱: 1.09s 

[SCRAPE].. ◆ https://books.toscrape.com/                                                                          |
✓ | ⏱: 0.03s 

[COMPLETE] ● https://books.toscrape.com/                                                                          |
✓ | ⏱: 1.13s 

...

📖 Crawling page 3: https://books.toscrape.com/catalogue/page-3.html


[FETCH]... ↓ https://books.toscrape.com/catalogue/page-3.html                                                     |
✓ | ⏱: 0.55s 

[SCRAPE].. ◆ https://books.toscrape.com/catalogue/page-3.html                                                     |
✓ | ⏱: 0.05s 

[COMPLETE] ● https://books.toscrape.com/catalogue/page-3.html                                                     |
✓ | ⏱: 0.61s 

✅ Saved 60 books from 3 pages to books_toscrape.csv
